# Regresja Logistyczna: wieloklasowość

Tematem dzisiejszych zajęć jest regresja logistyczna dla wielu (więcej niż 2) klas. Posłużymy się zbiorem danych [MNIST](http://yann.lecun.com/exdb/mnist/), który zawiera 50 tyś. obrazków cyfr. Naszym zadaniem będzie wytrenować model, na podstawie obrazka rozpozna jak to jest cyfra.

Zadania na dzisiejsze zajęcia są podzielone na 3 części: wczytanie i przygotowanie danych, implementacja regresji logistycznej i użycie modelu z pakietu `sklearn`. 

## Wczytanie i przygotowanie danych

**zad. 1** Pobierz cztery pliki `*-images-idx3-ubyte.gz` ze strony http://yann.lecun.com/exdb/mnist/ i rozpakuj (!) w tym samym katalogu, w którym znajduje się notatnik (pod Linuxem możesz wykorzystać skrypt `download.sh`).

By uprościć zadanie, zostały przygotowane dwie funkcje:
 * `readMnist`: funkcja wczytująca zestaw danych.
 * `showImage`: fukcja wyświetlająca pojedyńczy obrazek.

In [ ]:
import os
import struct
import numpy as np

def readMnist(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    return [get_img(i) for i in range(len(lbl))]

In [ ]:
def showImage(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

Wczytanie danych:

In [ ]:
train_data = readMnist()
valid_data = readMnist("testing")

Każdy przykład testowy składa się z dwóch elementów: cyfry oraz obrazu, który odpowiada tej cyfrze:

**zad. 2** Korzystając z fukcji `showImage` wyświetl pierwsze 5 przykładów z zbiory trenującego.

Każdy obrazek ma rozmiar 28 na 28 pikseli. Kazdy piksel będzie cechą dla naszego modelu regresji logistycznej (będziemy mieć 28 * 28 = 784 cech). Każdy piksel to liczba pomiedzy 0 a 256 (skala szarości). Ponadto będziemy meić 10 klas. Każda klasa będzie reprezentować cyfrę.

In [ ]:
train_data[0][1].shape

**zad. 3** Napisz funkcję, która przeskaluje dane wejściowe do przedziału od 0 do 1.

**zad. 4** Napisz fukcję, która rozdzieli `train_data` i `valid_data` na zbiory wejściowe i wyjściowe. 

In [ ]:
train_X = 
train_y = 

valid_X = 
valid_y = 

## Jeden kontra pozostałe

Na poprzednich zajęciach poznaliśmy regresję logistyczną, która działa na 2 klasach. Najprostszym podejściem jest klasyfikacja typu *One-vs-All*. Polega ona na zbudowaniu modelu modelu regresji logistycznej dla każdej klasy osobno. W naszym przypadku będzie to napisania modeli, które rozpoznają cyfry po kolei:0, 1, ..., 9.

Podczas testowania uruchmiamy wszystkie modele i sprawdzamy, który z nich ma najwyższe prawdopodobieństwo.

**zad. 5 (złożone)**

Zaimplementuj model *One-vs-All*:
 * Przygotuj 10 zestawów trenujących oraz testowych. Dla cyfry $i$, $i$-ty zestaw danych powienien mieć dwie klasy \[1, 0\].
 * Wytrenuj 10 modeli na wytrenowanych danych.
 * Przetestuj wszystkie modele osobno.
 * Przetestuj wszytkie modele jednocześnie. Dla każdego przypadku testowego, uruchom wszystkie 10 modeli. Przypisz indeks modelu z najwyższym prawdopodobieństem jako wyjście całego systemu.

## Implementacja regresji logistycznej

W tej cześci ćwiczeń skupimy się na ręcznej implementacji klasyfikatora regresji logistycznej. Na chwilę wrócimy do modelu z dwoma klasami.

In [ ]:
def batch_generator01(dataset = "training", path = ".", batch_size=32):
    ii = 0
    labels = []
    images = []
    for label, image in readMnist(dataset, path):
        if label in (0, 1):
            labels.append(label)
            images.append(image.reshape((28 * 28,)))
            ii += 1
            if ii == batch_size:
                yield np.array(images, dtype='float32') / 256.0, np.array(labels, dtype="int64")
                ii = 0
                labels = []
                images = []

**zad. 6** Zaimplementuj następujące funkcję:
 * `forward(x, weights, bias)`: oblicza $\sigma(dot(weights, x) + bias)$
 * `loss(labels, out)`: oblicza entropię krzyżową.
 * `grad_weights`: oblicza gradient wszystkich zmiennych z `weights` względem funkcji kosztu. Ponadto oblicza średnią po kolumnach.

Objaśnienia:
 * `dot`: iloczyn skalarny
 * `weights` jest listą długości 784 zawierającą wagi modelu
 * `x` jest wymiaru 32 na 784 (lista list)
 * `labels` jest listą etykiet (0 lub 1) rozmiaru 32
 * `y`: jest wyjściem z funkcji `forward`.

In [ ]:
import numpy as np

def sigma(x):
    return [1.0 / (1 + np.exp(-x_i)) for x_i in x]

def forward(x, weights, bias):
    pass

def grad_weights(out, x, y):
    grad_x = [0.0 for i in range(len(x[1]))]
    
    for x_row, y_i, out_i in zip(x, y, out):
        tmp = []
        for x_i in x_row:
            tmp.append((out_i - y_i) * x_i)
        for i in range(len(grad_x)):
            grad_x[i] += tmp[i]
            
    for i in range(len(grad_x)):
        grad_x[i] /= len(y)
    
    return grad_x  

def grad_bias(out, y):
    return sum([out_i - y_i for out_i, y_i in zip(out, y)]) / len(out)

def loss(y, out):
    pass

def update_weights(weights, bias, grads, lr):
    pass

    
def train_logistic_regression(epochs, batch_generator, lr=0.01):
    weights = np.random.normal(0, 0.05, 28 * 28)
    bias = 0.0
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch in batch_generator():
            x = batch[0]
            y = batch[1]
            
            out = forward(x, weights, bias)

            epoch_loss = sum(loss(y, out)) / len(y)
            print("loss", epoch_loss)
            grads = (grad_weights(out, x, y), grad_bias(out, y))
            
            weights, bias = update_weights(weights, bias, grads, lr)
    return weights, bias    

Zrobiwszy możemy uruchomić model:

In [ ]:
weights, bias = train_logistic_regression(1, batch_generator01)

Sprawdzamy, na których przykładach, nasz model pomylił się:

In [ ]:
%matplotlib inline
for e in batch_generator01('testing', '.', 1):
    img = e[0]
    label = e[1][0]
    if int(forward(img, weights, bias)[0] >= 0.5) != label:
        showImage(img.reshape((28,28)))

## Wieloklasowa regresja logistyczna (albo regresja softmax)

**zad. 7** Zaimplementuj funckję [``softmax``](https://en.wikipedia.org/wiki/Softmax_function):

$$ softmax(x_i) = \frac{e^{x_i}}{\sum{e^{x_j}}} $$


Przykład:

wejście:

``
[[0.2, 0.3, 0.1],
 [1.0, 2.0, -3]]``

poprawna odpowiedź:

``
[[0.3322249935333472, 0.36716540111092544, 0.3006096053557273],
 [0.2676231541498623, 0.7274751568004647, 0.004901689049672921]]``

In [ ]:
def softmax(matrix):
    for row in matrix:
        row_sum = 0.0
        for col in row:
            row_sum += np.exp(col)
        
        for i in range(len(row)):
            row[i] = np.exp(row[i]) / row_sum
    return matrix

Importujemy ten sam model jak w przypadku regresji 2-klasowej:

In [ ]:
from sklearn.linear_model import LogisticRegression

Musimy ustawić opcję `multi_class` na `multinomial` i `solver` na `saga`:

In [ ]:
model = LogisticRegression(multi_class='multinomial', penalty='l1', solver='saga', tol=0.1)

Trening modelu nie różni się od treningu modelu:

In [ ]:
model.fit([x[1].flatten() for x in readMnist()], [x[0] for x in readMnist()])

Predykcja również zostaje niezmieniona:

In [ ]:
predicted = model.predict([x[1].flatten() for x in readMnist('testing')])
y_test = [x[0] for x in readMnist('testing')]

from sklearn.metrics import accuracy_score

accuracy_score(predicted, y_test)

Jeżeli dotarłeś do tego miejsca podczas zajeć: Gratuluję! Możesz albo skończyć zajęcia, ale zachęcam Cię do jeszcze jednego zadania.

**zad. dodatkowe** Przekształć kod z regresji dwuklasowej, żeby działał z regresją softmax.